<a href="https://colab.research.google.com/github/ljh4697/joonhyeok/blob/master/hello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
# expert 버젼


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets

input_shape = (28, 28, 1)
num_classes = 10

inputs = layers.Input(input_shape, dtype=tf.float32)
net = layers.Conv2D(32, (32, 32), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (32, 32), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.5)(net)

net = layers.Conv2D(64, (32, 32), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (32, 32), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.5)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

mnist = tf.keras.datasets.mnist
    # load data from Mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

    #channel 차원 추가
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

    #data Normalization
x_train, x_test = x_train / 255.0 , x_test / 255.0

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(1000)  # 버퍼 사이즈 1000
train_ds = train_ds.batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
# 여기서는 굳이 셔플안해도됨
test_ds = test_ds.batch(32)

import matplotlib.pyplot as plt
# image, label = next(iter(train_ds))
# print(image.shape, label.shape)

# plt.title(label[0])
# plt.imshow(image[0, :, :, 0] , 'gray')
# plt.show()
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
# model.fit(train_ds, epochs=10)


In [148]:
 # expert 한 버젼
 loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

 optimizer = tf.keras.optimizers.Adam()


In [149]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
# 에폭마다 loss 의 평균을 재서 보여줌
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


# **Trainning**
@ tf.function - 기존 session 열었던 것처럼 바로 작동 안하고 , 그래프만 만들고 학습이 시작되면 돌아가도록 함

In [150]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [151]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)
  
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [152]:
for epoch in range(2):
  print('start Trainning')
  for lmages, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)
  
  template = 'Epoch {}, Loss: {}, Accuracy : {}, Test Loss {}, Test Accuracy : {}'
  print(template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))


start Trainning
Epoch 1, Loss: 2.3039586544036865, Accuracy : 11.210000038146973, Test Loss 2.3012454509735107, Test Accuracy : 11.350000381469727
start Trainning
Epoch 2, Loss: 2.3027124404907227, Accuracy : 11.223333358764648, Test Loss 2.301234483718872, Test Accuracy : 11.350000381469727


##gfg


In [153]:
rh

NameError: ignored